# Unfolding Long Exposure

Creates long exposure images/videos by accumulating frames from a sequence of photos using thresholding to select pixels.


In [1]:
import cv2
import numpy as np
import os
import time
from pathlib import Path


## Configuration

Adjust all settings here before running the processing steps.


In [4]:
# ===== INPUT SETTINGS =====
INPUT_FOLDER = 'photos/photos_andrea_and_jager_creature'
IMAGE_TYPE = 'tif'  # File extension without dot (e.g., 'tif', 'jpg', 'png')

# ===== THRESHOLD SETTINGS =====
THRESHOLD = 4  # Pixels above this value (0-255) will be included in accumulation
THRESHOLD_KERNEL_SIZE = (7, 7)  # Gaussian blur kernel size for thresholding

# ===== EXPOSURE ADJUSTMENT SETTINGS =====
PREVIEW_GAMMA = 1.0  # Gamma for preview during exposure adjustment
VISUALIZE_ADJUSTMENT = False  # Show preview window during adjustment
USE_MAXIMUM_FOR_RATIO = True  # If True, uses max of entire image; if False, lets you select ROI
DEEXPOSURE_RATIO = None  # If None, will be calculated. Otherwise, use pre-calculated value.

# ===== RENDER SETTINGS =====
RENDER_GAMMA = 1.0  # Final gamma correction for output
SHOW_MOMENT = True  # Show fade-out effect in video
FADE_OUT_FRAMES = 50  # Number of frames for fade-out effect
VIDEO_FPS = 25  # Frames per second for video output

# ===== OUTPUT SETTINGS =====
OUTPUT_NAME = None  # If None, will be auto-generated from timestamp
OUTPUT_DIR = 'results'  # Directory for output files


## Class Definition


In [2]:
class UnfoldingLongExposure:
    """Creates unfolding long exposure images/videos from a sequence of photos."""
    
    def __init__(self, input_folder, output_file=None, threshold_kernel_size=(7, 7), 
                 threshold=2, image_type='tif', deexposure_ratio=None, output_dir='results'):
        """
        Initialize the long exposure processor.
        
        Args:
            input_folder: Path to folder containing input images
            output_file: Optional name for output file
            threshold_kernel_size: Gaussian blur kernel size for thresholding
            threshold: Threshold value (0-255) for pixel selection
            image_type: Image file extension without dot
            deexposure_ratio: Pre-calculated deexposure ratio (None to calculate)
            output_dir: Directory for output files
        """
        # Ensure folder path ends with /
        input_folder = str(Path(input_folder))
        if not input_folder.endswith(os.sep):
            input_folder += os.sep
            
        self.folder = input_folder
        self.output = output_file
        self.threshold_kernel_size = threshold_kernel_size
        self.threshold = threshold
        self.image_type = '.' + image_type
        self.deexposure_ratio = deexposure_ratio
        self.output_dir = output_dir
        
        # Ensure output directory exists
        Path(self.output_dir).mkdir(parents=True, exist_ok=True)
    
    def get_threshold_binary(self, image):
        """Create binary mask using thresholding."""
        # Convert to grayscale and blur
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, self.threshold_kernel_size, 0)
        
        # Apply threshold: pixels > threshold become white (255), others black (0)
        _, thresh = cv2.threshold(blurred, self.threshold, 255, cv2.THRESH_BINARY)
        return thresh
    
    def adjust_gamma(self, image, gamma):
        """Apply gamma correction to image."""
        if gamma == 1.0:
            return image
            
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255
                         for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(image, table)
    
    def process_frame(self, image, gamma=1.0):
        """Process a single frame: threshold, gamma adjust, and mask."""
        binary = self.get_threshold_binary(image)
        adjusted = self.adjust_gamma(image, gamma)
        masked = cv2.bitwise_and(adjusted, adjusted, mask=binary)
        return masked
    
    def set_deexposure_ratio(self, int8_image, int32_image, to_maximum=False):
        """Calculate deexposure ratio to normalize accumulated values to 0-255 range."""
        if not to_maximum:
            # Let user select ROI
            print("Select ROI in the preview window, then press SPACE or ENTER to confirm, or ESC to cancel")
            r = cv2.selectROI("Select ROI for deexposure calculation", int8_image)
            if r[2] > 0 and r[3] > 0:  # Valid selection
                int32_crop = int32_image[int(r[1]):int(r[1]+r[3]), 
                                        int(r[0]):int(r[0]+r[2])]
                self.deexposure_ratio = 255 / int32_crop.max()
            else:
                # Fallback to maximum if ROI selection cancelled
                self.deexposure_ratio = 255 / int32_image.max()
        else:
            self.deexposure_ratio = 255 / int32_image.max()
        
        print(f'Deexposure Ratio: {self.deexposure_ratio}')
        cv2.destroyAllWindows()
    
    def adjust_exposure(self, preview_gamma=1.0, visualize=False, to_maximum=False):
        """
        Process all images to calculate deexposure ratio.
        
        Args:
            preview_gamma: Gamma correction for preview visualization
            visualize: Show preview window during processing
            to_maximum: Use max of entire image instead of ROI selection
        """
        int8_image = None
        int32_image = None
        
        image_files = sorted([f for f in os.listdir(self.folder) 
                             if f.endswith(self.image_type)])
        
        if not image_files:
            raise ValueError(f"No {self.image_type} files found in {self.folder}")
        
        print(f"Processing {len(image_files)} images...")
        
        for filename in image_files:
            image_path = os.path.join(self.folder, filename)
            image = cv2.imread(image_path)
            
            if image is None:
                print(f"Warning: Could not read {filename}")
                continue
            
            # Process with preview gamma (for visualization) and without (for calculation)
            int8_masked = self.process_frame(image, preview_gamma)
            int32_masked = self.process_frame(image)
            
            if int8_image is None:
                int8_image = np.asarray(int8_masked, dtype="int32")
                int32_image = np.asarray(int32_masked, dtype="int32")
            else:
                int8_image += np.asarray(int8_masked, dtype="int32")
                int8_image = int8_image.clip(0, 255)
                int32_image += np.asarray(int32_masked, dtype="int32")
            
            print(f"  Processed: {filename}")
            
            if visualize:
                cv2.imshow('Accumulation Preview', int8_image.astype(np.uint8))
                cv2.waitKey(1)
        
        if visualize:
            cv2.destroyWindow('Accumulation Preview')
        
        self.set_deexposure_ratio(int8_image.astype(np.uint8), int32_image, to_maximum)
    
    def render_image(self, gamma=1.0):
        """Render final long exposure image."""
        if self.deexposure_ratio is None:
            raise ValueError("Deexposure ratio not set. Run adjust_exposure() first.")
        
        final_image = None
        image_files = sorted([f for f in os.listdir(self.folder) 
                             if f.endswith(self.image_type)])
        
        print(f"Rendering image from {len(image_files)} frames...")
        
        for filename in image_files:
            image_path = os.path.join(self.folder, filename)
            image = cv2.imread(image_path)
            
            if image is None:
                continue
            
            masked = self.process_frame(image)
            
            if final_image is None:
                final_image = np.asarray(masked, dtype="int32")
            else:
                final_image += np.asarray(masked, dtype="int32")
            
            print(f"  Processed: {filename}")
        
        # Apply deexposure ratio and gamma correction
        exposed = (final_image * self.deexposure_ratio).clip(0, 255).astype(np.uint8)
        final = self.adjust_gamma(exposed, gamma)
        
        # Save image
        timestamp = int(time.time())
        output_path = os.path.join(self.output_dir, f'long_exposure_{timestamp}{self.image_type}')
        cv2.imwrite(output_path, final)
        print(f"\nSaved: {output_path}")
        
        return final
    
    def render_video(self, gamma=1.0, show_moment=True, fade_out_frames=3, 
                     fps=25, save_frames=False):
        """
        Render unfolding long exposure video.
        
        Args:
            gamma: Final gamma correction
            show_moment: Show fade-out effect for recent frames
            fade_out_frames: Number of frames in fade-out buffer
            fps: Frames per second for video
            save_frames: If True, save individual frames; if False, save video
        """
        if self.deexposure_ratio is None:
            raise ValueError("Deexposure ratio not set. Run adjust_exposure() first.")
        
        final_image = None
        fade_out_buffer = []
        final_frames = []
        
        # Calculate moment intensity for fade-out effect
        fade_sum = sum((i+1) / (fade_out_frames+1) for i in range(fade_out_frames))
        moment_intensity = 1 / fade_sum if fade_sum > 0 else 1
        
        image_files = sorted([f for f in os.listdir(self.folder) 
                             if f.endswith(self.image_type)])
        
        print(f"Rendering video from {len(image_files)} frames...")
        
        for filename in image_files:
            image_path = os.path.join(self.folder, filename)
            image = cv2.imread(image_path)
            
            if image is None:
                continue
            
            if final_image is None:
                # Initialize with same shape as processed frame (may be grayscale or color)
                sample_masked = self.process_frame(image)
                final_image = np.zeros(sample_masked.shape, dtype='int32')
            
            masked = np.asarray(self.process_frame(image), dtype="int32")
            fade_out_buffer.append(masked.copy())
            
            # Move oldest frame from buffer to accumulation
            if len(fade_out_buffer) > fade_out_frames:
                final_image += fade_out_buffer.pop(0)
            
            # Create frame: accumulated image + fade-out effect
            frame = final_image.copy()
            exposed = (frame * self.deexposure_ratio)
            
            if show_moment:
                for i, fader in enumerate(fade_out_buffer):
                    weight = ((i+1) / (fade_out_frames+1)) * moment_intensity
                    exposed += weight * fader
            
            gammad = self.adjust_gamma(exposed, gamma)
            final_frames.append(gammad.clip(0, 255).astype(np.uint8))
            print(f"  Processed: {filename}")
        
        # Handle remaining fade-out frames
        if show_moment:
            while fade_out_buffer:
                frame = final_image.copy()
                exposed = (frame * self.deexposure_ratio)
                for i, fader in enumerate(fade_out_buffer):
                    weight = ((i+1) / (len(fade_out_buffer)+1)) * moment_intensity
                    exposed += weight * fader
                gammad = self.adjust_gamma(exposed, gamma)
                final_frames.append(gammad.clip(0, 255).astype(np.uint8))
                final_image += fade_out_buffer.pop(0)
        
        # Generate output name
        timestamp = int(time.time())
        name_parts = [str(timestamp)]
        if self.output:
            name_parts.append(str(self.output))
        name_parts.append(f"ratio{self.deexposure_ratio:.6f}")
        name = '_'.join(name_parts)
        
        if save_frames:
            # Save individual frames
            directory = os.path.join(self.output_dir, name)
            os.makedirs(directory, exist_ok=True)
            for i, frame in enumerate(final_frames):
                cv2.imwrite(os.path.join(directory, f'{1000+i:04d}.tif'), frame)
            print(f"\nSaved {len(final_frames)} frames to: {directory}")
        else:
            # Save video
            height, width = final_frames[0].shape[:2]
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_path = os.path.join(self.output_dir, f'{name}.mp4')
            video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))
            
            for frame in final_frames:
                video_writer.write(frame)
            
            video_writer.release()
            print(f"\nSaved video: {video_path}")
        
        return final_frames


In [5]:
# Create processor instance
processor = UnfoldingLongExposure(
    input_folder=INPUT_FOLDER,
    output_file=OUTPUT_NAME,
    threshold_kernel_size=THRESHOLD_KERNEL_SIZE,
    threshold=THRESHOLD,
    image_type=IMAGE_TYPE,
    deexposure_ratio=DEEXPOSURE_RATIO,
    output_dir=OUTPUT_DIR
)


In [7]:
# Calculate deexposure ratio
processor.adjust_exposure(
    preview_gamma=PREVIEW_GAMMA,
    visualize=VISUALIZE_ADJUSTMENT,
    to_maximum=USE_MAXIMUM_FOR_RATIO
)

# The calculated ratio is now stored in processor.deexposure_ratio
print(f"\nDeexposure ratio: {processor.deexposure_ratio}")


Processing 2575 images...
  Processed: 10000.tif
  Processed: 10001.tif
  Processed: 10002.tif
  Processed: 10003.tif
  Processed: 10004.tif
  Processed: 10005.tif
  Processed: 10006.tif
  Processed: 10007.tif
  Processed: 10008.tif
  Processed: 10009.tif
  Processed: 10010.tif
  Processed: 10011.tif
  Processed: 10012.tif
  Processed: 10013.tif
  Processed: 10014.tif
  Processed: 10015.tif
  Processed: 10016.tif
  Processed: 10017.tif
  Processed: 10018.tif
  Processed: 10019.tif
  Processed: 10020.tif
  Processed: 10021.tif
  Processed: 10022.tif
  Processed: 10023.tif
  Processed: 10024.tif
  Processed: 10025.tif
  Processed: 10026.tif
  Processed: 10027.tif
  Processed: 10028.tif
  Processed: 10029.tif
  Processed: 10030.tif
  Processed: 10031.tif
  Processed: 10032.tif
  Processed: 10033.tif
  Processed: 10034.tif
  Processed: 10035.tif
  Processed: 10036.tif
  Processed: 10037.tif
  Processed: 10038.tif
  Processed: 10039.tif
  Processed: 10040.tif
  Processed: 10041.tif
  Proces

## Step 2: Render Output

Choose one of the rendering options below.


### Option A: Render Single Image


In [ ]:
# Render final long exposure image
final_image = processor.render_image(gamma=RENDER_GAMMA)

# Display the result (optional)
from matplotlib import pyplot as plt
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Long Exposure Result')
plt.show()


Rendering image from 2575 frames...
  Processed: 10000.tif
  Processed: 10001.tif
  Processed: 10002.tif
  Processed: 10003.tif
  Processed: 10004.tif
  Processed: 10005.tif
  Processed: 10006.tif
  Processed: 10007.tif
  Processed: 10008.tif
  Processed: 10009.tif
  Processed: 10010.tif
  Processed: 10011.tif
  Processed: 10012.tif
  Processed: 10013.tif
  Processed: 10014.tif
  Processed: 10015.tif
  Processed: 10016.tif
  Processed: 10017.tif
  Processed: 10018.tif
  Processed: 10019.tif
  Processed: 10020.tif
  Processed: 10021.tif
  Processed: 10022.tif
  Processed: 10023.tif
  Processed: 10024.tif
  Processed: 10025.tif
  Processed: 10026.tif
  Processed: 10027.tif
  Processed: 10028.tif
  Processed: 10029.tif
  Processed: 10030.tif
  Processed: 10031.tif
  Processed: 10032.tif
  Processed: 10033.tif
  Processed: 10034.tif
  Processed: 10035.tif
  Processed: 10036.tif
  Processed: 10037.tif
  Processed: 10038.tif
  Processed: 10039.tif
  Processed: 10040.tif
  Processed: 10041.ti

ModuleNotFoundError: No module named 'matplotlib'

### Option B: Render Video (Unfolding Effect)


In [9]:
# Render unfolding video
frames = processor.render_video(
    gamma=RENDER_GAMMA,
    show_moment=SHOW_MOMENT,
    fade_out_frames=FADE_OUT_FRAMES,
    fps=VIDEO_FPS,
    save_frames=False  # Set to True to save individual frames instead of video
)


Rendering video from 2575 frames...
  Processed: 10000.tif
  Processed: 10001.tif
  Processed: 10002.tif
  Processed: 10003.tif
  Processed: 10004.tif
  Processed: 10005.tif
  Processed: 10006.tif
  Processed: 10007.tif
  Processed: 10008.tif
  Processed: 10009.tif
  Processed: 10010.tif
  Processed: 10011.tif
  Processed: 10012.tif
  Processed: 10013.tif
  Processed: 10014.tif
  Processed: 10015.tif
  Processed: 10016.tif
  Processed: 10017.tif
  Processed: 10018.tif
  Processed: 10019.tif
  Processed: 10020.tif
  Processed: 10021.tif
  Processed: 10022.tif
  Processed: 10023.tif
  Processed: 10024.tif
  Processed: 10025.tif
  Processed: 10026.tif
  Processed: 10027.tif
  Processed: 10028.tif
  Processed: 10029.tif
  Processed: 10030.tif
  Processed: 10031.tif
  Processed: 10032.tif
  Processed: 10033.tif
  Processed: 10034.tif
  Processed: 10035.tif
  Processed: 10036.tif
  Processed: 10037.tif
  Processed: 10038.tif
  Processed: 10039.tif
  Processed: 10040.tif
  Processed: 10041.ti

### Option C: Quick Workflow (Skip Ratio Calculation)

If you already know the deexposure ratio, you can set it directly and render:


In [ ]:
# Example: Use a pre-calculated ratio
# processor.deexposure_ratio = 0.002232729183083793
# final_image = processor.render_image(gamma=RENDER_GAMMA)
# OR
# frames = processor.render_video(gamma=RENDER_GAMMA, show_moment=SHOW_MOMENT, fade_out_frames=FADE_OUT_FRAMES)
